In [11]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
from matplotlib import font_manager as fm
import networks
import mne_rsa
import editdistance
from pilot import utils
from scipy.spatial import distance
import seaborn as sns
import pickle
from tqdm import tqdm
from io import BytesIO
import os
import contextlib

In [12]:
def make_image(word, rotation=0, size=16, family='dejavu sans', fname=None, noise=0):
    # Create figure of exactly 64x64 pixels
    dpi = 96.
    f = Figure(figsize=(64 / dpi, 64 / dpi), dpi=dpi)
    
    # Initialize an empty figure
    f.clf()
    ax = f.add_axes([0, 0, 1, 1])
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    # Start with a grey background image
    background = plt.Rectangle((0, 0), 64, 64, facecolor=(0.5, 0.5, 0.5, 1.0), zorder=0)
    ax.add_patch(background)

    # Add noise to the image. Note the usage of the alpha parameter to tweak the amount of noise
    noise_image = np.random.randn(64, 64)
    ax.imshow(noise_image, extent=[0, 1, 0, 1], cmap='gray', alpha=noise, zorder=1)
    
    # Add the text to the image in the selected font
    fontprop = fm.FontProperties(family=family, fname=fname, size=size)
    ax.text(0.5, 0.5, word, ha='center', va='center',
            rotation=rotation, fontproperties=fontprop, alpha=1 - noise, zorder=2)

    # We need the canvas object to get the bitmap data at the end
    canvas = FigureCanvasAgg(f)
    
    # Render the image and create a PIL.Image from the pixel data
    canvas.draw()
    buffer, (width, height) = canvas.print_to_buffer()
    image = np.frombuffer(buffer, np.uint8).reshape((height, width, 4))[:, :, :3]
    image = Image.fromarray(image)
    
    # The preprocessing transform used during training of the model
    preproc = transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(60),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.2, 0.2, 0.2]),
        ])
    
    # Transform the PIL image to a PyTorch tensor for feeding into the model
    image = preproc(image).unsqueeze(0)
    return image

# Create images with noises and see how CNN model gets activated

In [13]:
# Construct images
images_noise = []
for noise in np.arange(0, 1.0, 0.01):
    images_noise.append(make_image(word='odote', noise=noise))
images_noise = torch.cat(images_noise, 0)

# Plot the images
plt.figure(figsize=(10, 10))
plt.axis('off')
plt.tight_layout()
plt.imshow(make_grid(images_noise/5 + 0.5, nrow=10).numpy().transpose(1, 2, 0));

### Get DSM

In [65]:
# Load the model and feed through the images
model_name = 'vgg_first_imagenet64_then_tiny-words_tiny-nontext_tiny-imagenet_w2v'
checkpoint = torch.load('../models/%s.pth.tar' % model_name, map_location='cpu')
model = networks.vgg_small.from_checkpoint(checkpoint)
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    feature_outputs, classifier_outputs, semantic_outputs = model.get_layer_activations(images_noise)

# calculate DSMs
dsms_values = [
    mne_rsa.compute_dsm(feature_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[3], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(semantic_outputs[0], metric='correlation'),
]

fig, axes = plt.subplots(2, 5, figsize=(14, 7))
ticklabels = [0, 67, 99]

for i in range(len(dsms_values)):
    dsm = dsms_values[i]
    ax = axes[i%2, i//2]
    if i < len(dsms_values)-1:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    else:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar_ax=axes[1, 4], square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    ax.set_xticks(ticklabels)
    ax.set_yticks(ticklabels)

### Get RSA with pixel-wise DSM

In [66]:
DSM_pixelwise_noise = mne_rsa.compute_dsm(images_noise, metric='euclidean')
mne_rsa.plot_dsms(DSM_pixelwise_noise);

In [67]:
rsa_noise = mne_rsa.rsa(dsms_values, DSM_pixelwise_noise, metric='kendall-tau-a')
print(rsa_noise)

[0.90977318 0.65585081 0.61111817 0.56922876 0.56614159 0.56313362
 0.32551818 0.25493415]


### Where Are Noisy Images Classified?

In [25]:
plt.imshow(classifier_outputs[-1]);

# Try the Same Thing for Different Fonts

In [30]:
def images_with_fonts(words, fonts, font_size=16):
    images = []
    
    for word in words:
        for font in fonts:
            fontfamily, fontfile = fonts[font]
            image = make_image(word, rotation=0, size=font_size, family=fontfamily, fname=fontfile, noise=0)
            images.append(image)
          
    return images

In [33]:
fonts = {
        'ubuntu mono': [None, '../data/fonts/UbuntuMono-R.ttf'],
        'courier': [None, '../data/fonts/courier.ttf'],
        'luxi mono regular': [None, '../data/fonts/luximr.ttf'],
        'lucida console': [None, '../data/fonts/LucidaConsole-R.ttf'],
        'lekton': [None, '../data/fonts/Lekton-Regular.ttf'],
        'dejavu sans mono': [None, '../data/fonts/DejaVuSansMono.ttf'],
        'times new roman': [None, '../data/fonts/times.ttf'],
        'arial': [None, '../data/fonts/arial.ttf'],
        'arial black': [None, '../data/fonts/arialbd.ttf'],
        'verdana': [None, '../data/fonts/verdana.ttf'],
        'comic sans ms': [None, '../data/fonts/comic.ttf'],
        'georgia': [None, '../data/fonts/georgia.ttf'],
        'liberation serif': [None, '../data/fonts/LiberationSerif-Regular.ttf'],
        'impact': [None, '../data/fonts/impact.ttf'],
        'roboto condensed': [None, '../data/fonts/Roboto-Light.ttf'],
    }
images_font = torch.cat(images_with_fonts(['odote'], fonts, font_size=12), 0)

In [34]:
# Plot the images
plt.figure(figsize=(10, 2))
plt.axis('off')
plt.tight_layout()
plt.imshow(make_grid(images_font/5 + 0.5, nrow=10).numpy().transpose(1, 2, 0));

### Check DSM

In [37]:
# Load the model and feed through the images
checkpoint = torch.load('../models/%s.pth.tar' % model_name, map_location='cpu')
model = networks.vgg_small.from_checkpoint(checkpoint)
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    feature_outputs, classifier_outputs, semantic_outputs = model.get_layer_activations(images_font)

# calculate DSMs
dsms_values = [
    mne_rsa.compute_dsm(feature_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[3], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(semantic_outputs[0], metric='correlation'),
]

fig, axes = plt.subplots(2, 4, figsize=(14, 7))
ticklabels = []

for i in range(len(dsms_values)):
    dsm = dsms_values[i]
    ax = axes[i%2, i//2]
    if i < len(dsms_values)-1:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    else:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    ax.set_xticks(ticklabels)
    ax.set_yticks(ticklabels)

### Find DSM for the raw images (over different fonts)

In [38]:
DSM_pixelwise_font = mne_rsa.compute_dsm(images_font, metric='euclidean')
mne_rsa.plot_dsms(DSM_pixelwise_font);

In [39]:
rsa_font = mne_rsa.rsa(dsms_values, DSM_pixelwise_font, metric='kendall-tau-a')
print(rsa_font)

[0.60842491 0.51428571 0.47985348 0.53333333 0.28681319 0.20695971
 0.25677656 0.25622711]


# Do it again, for rotation!

In [40]:
rotations = range(0, 360, 30)

words = ["odote"]

images_rot = []
for word in words:
    for rot in rotations:
        image_rot = make_image(word, rotation=rot, size=12, noise=0)
        images_rot.append(image_rot)
images_rot = torch.cat(images_rot, 0)

In [41]:
# Plot the images
plt.figure(figsize=(10, 2))
plt.axis('off')
plt.tight_layout()
plt.imshow(make_grid(images_rot/5 + 0.5, nrow=10).numpy().transpose(1, 2, 0));

In [42]:
# Load the model and feed through the images
model_name = 'vgg_first_imagenet64_then_tiny-words_tiny-nontext_tiny-imagenet_w2v'
checkpoint = torch.load('../models/%s.pth.tar' % model_name, map_location='cpu')
model = networks.vgg_small.from_checkpoint(checkpoint)
feature_outputs, classifier_outputs, semantic_outputs = model.get_layer_activations(images_rot)

feature layer 00, output=torch.Size([12, 64, 60, 60])
feature layer 01, output=torch.Size([12, 64, 60, 60])
feature layer 02, output=torch.Size([12, 64, 60, 60])
feature layer 03, output=torch.Size([12, 64, 60, 60])
feature layer 04, output=torch.Size([12, 64, 60, 60])
feature layer 05, output=torch.Size([12, 64, 60, 60])
feature layer 06, output=torch.Size([12, 64, 30, 30])
feature layer 07, output=torch.Size([12, 128, 30, 30])
feature layer 08, output=torch.Size([12, 128, 30, 30])
feature layer 09, output=torch.Size([12, 128, 30, 30])
feature layer 10, output=torch.Size([12, 128, 30, 30])
feature layer 11, output=torch.Size([12, 128, 30, 30])
feature layer 12, output=torch.Size([12, 128, 30, 30])
feature layer 13, output=torch.Size([12, 128, 15, 15])
feature layer 14, output=torch.Size([12, 256, 15, 15])
feature layer 15, output=torch.Size([12, 256, 15, 15])
feature layer 16, output=torch.Size([12, 256, 15, 15])
feature layer 17, output=torch.Size([12, 256, 15, 15])
feature layer 18,

In [43]:
dsms_values = [
    mne_rsa.compute_dsm(feature_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[3], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(semantic_outputs[0], metric='correlation'),
]

fig, axes = plt.subplots(2, 4, figsize=(14, 7))
ticklabels = []

for i in range(len(dsms_values)):
    dsm = dsms_values[i]
    ax = axes[i%2, i//2]
    if i < len(dsms_values)-1:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    else:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    ax.set_xticks(ticklabels)
    ax.set_yticks(ticklabels)

In [44]:
DSM_pixelwise_rot = mne_rsa.compute_dsm(images_rot, metric='euclidean')
mne_rsa.plot_dsms(DSM_pixelwise_rot);

In [45]:
rsa_rot = mne_rsa.rsa(dsms_values, DSM_pixelwise_rot, metric='kendall-tau-a')
print(rsa_rot)

[0.80885781 0.72027972 0.65780886 0.65874126 0.53659674 0.38181818
 0.10862471 0.08438228]


# Lastly, try it for word length!

In [48]:
meta = utils.get_stimulus_info(subject=2, data_path='../data')
meta.info()

Reading ../data/pilot_data/pilot2/pilot2_epo.fif ...
    Read a total of 13 projection items:
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
        generated with autossp-1.0.1 (1 x 306) active
    Found the data of interest:
        t =    -200.00 ...    1000.00 ms
        0 CTF compensation matrices available
Reading ../data/pilot_data/pilot2\pilot2_epo-1.fif ...
    Read a total of 13 project

,type,freq,font,fontsize,rotation,noise_level,filename,question,question_correct,question_filename,event_id,question_asked,y
text,,,,,,,,,,,,,
odote,word,20.0,Comic Sans MS,20.0,0.0,0.5,word_odote.png,_ _ _ _ j,False,word_odote_question.png,10.0,False,0
nurin,word,1001.0,Times New Roman,20.0,-15.0,0.5,word_nurin.png,_ _ _ _ l,False,word_nurin_question.png,10.0,False,1
nurja,word,11.0,Times New Roman,30.0,-15.0,0.5,word_nurja.png,_ _ _ _ a,True,word_nurja_question.png,10.0,False,2
näkö,word,506.0,Times New Roman,30.0,-15.0,0.5,word_näkö.png,n _ _ _,True,word_näkö_question.png,10.0,False,3
näppy,word,15.0,Impact,30.0,0.0,0.5,word_näppy.png,_ _ u _ _,False,word_näppy_question.png,10.0,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ds^^o,symbols,NaN,DejaVu Sans,40.0,0.0,0.5,symbols_ds^^o.png,_ s _ _ _,True,symbols_ds^^o_question.png,30.0,False,355
dsdoo^,symbols,NaN,DejaVu Sans,30.0,0.0,0.5,symbols_dsdoo^.png,_ _ _ v _ _,False,symbols_dsdoo^_question.png,30.0,False,356
dsovsd,symbols,NaN,DejaVu Sans,40.0,15.0,0.2,symbols_dsovsd.png,_ _ _ _ _ ^,False,symbols_dsovsd_question.png,30.0,False,357


In [51]:
meta['length'] = meta.index.str.len()
meta.query('type=="word"').sort_values('length', ascending=False).head(5)

,type,freq,font,fontsize,rotation,noise_level,filename,question,question_correct,question_filename,event_id,question_asked,y,length
text,,,,,,,,,,,,,,
tuohus,word,5.0,Comic Sans MS,20.0,0.0,0.20,word_tuohus.png,_ _ _ u _ _,False,word_tuohus_question.png,10.0,False,90,6
särkyä,word,94.0,Impact,20.0,15.0,0.50,word_särkyä.png,j _ _ _ _ _,False,word_särkyä_question.png,10.0,False,120,6
syaani,word,10.0,Times New Roman,20.0,-15.0,0.50,word_syaani.png,s _ _ _ _ _,True,word_syaani_question.png,10.0,True,113,6
suippo,word,26.0,Times New Roman,40.0,-15.0,0.35,word_suippo.png,_ _ _ p _ _,True,word_suippo_question.png,10.0,True,111,6
stoola,word,8.0,Impact,20.0,15.0,0.20,word_stoola.png,_ t _ _ _ _,True,word_stoola_question.png,10.0,False,110,6


In [52]:
images_len=[]
for word in ['t', 'tu', 'tuo', 'tuoh', 'tuohu', 'tuohus']:
    image_len = make_image(word, rotation=0, size=12, noise=0)
    images_len.append(image_len)
images_len = torch.cat(images_len, 0)

In [54]:
# Plot the images
plt.figure(figsize=(5, 1))
plt.axis('off')
plt.tight_layout()
plt.imshow(make_grid(images_len/5 + 0.5, nrow=6).numpy().transpose(1, 2, 0));

In [56]:
# Load the model and feed through the images
model_name = 'vgg_first_imagenet64_then_tiny-words_tiny-nontext_tiny-imagenet_w2v'
checkpoint = torch.load('../models/%s.pth.tar' % model_name, map_location='cpu')
model = networks.vgg_small.from_checkpoint(checkpoint)
feature_outputs, classifier_outputs, semantic_outputs = model.get_layer_activations(images_len)

feature layer 00, output=torch.Size([6, 64, 60, 60])
feature layer 01, output=torch.Size([6, 64, 60, 60])
feature layer 02, output=torch.Size([6, 64, 60, 60])
feature layer 03, output=torch.Size([6, 64, 60, 60])
feature layer 04, output=torch.Size([6, 64, 60, 60])
feature layer 05, output=torch.Size([6, 64, 60, 60])
feature layer 06, output=torch.Size([6, 64, 30, 30])
feature layer 07, output=torch.Size([6, 128, 30, 30])
feature layer 08, output=torch.Size([6, 128, 30, 30])
feature layer 09, output=torch.Size([6, 128, 30, 30])
feature layer 10, output=torch.Size([6, 128, 30, 30])
feature layer 11, output=torch.Size([6, 128, 30, 30])
feature layer 12, output=torch.Size([6, 128, 30, 30])
feature layer 13, output=torch.Size([6, 128, 15, 15])
feature layer 14, output=torch.Size([6, 256, 15, 15])
feature layer 15, output=torch.Size([6, 256, 15, 15])
feature layer 16, output=torch.Size([6, 256, 15, 15])
feature layer 17, output=torch.Size([6, 256, 15, 15])
feature layer 18, output=torch.Size

In [57]:
dsms_values = [
    mne_rsa.compute_dsm(feature_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(feature_outputs[3], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[0], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[1], metric='correlation'),
    mne_rsa.compute_dsm(classifier_outputs[2], metric='correlation'),
    mne_rsa.compute_dsm(semantic_outputs[0], metric='correlation'),
]

fig, axes = plt.subplots(2, 4, figsize=(14, 7))
ticklabels = []


for i in range(len(dsms_values)):
    dsm = dsms_values[i]
    ax = axes[i%2, i//2]
    if i < len(dsms_values)-1:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    else:
        sns.heatmap(distance.squareform(dsm), ax=ax, cbar=False, square=True, xticklabels=ticklabels,
                   yticklabels=ticklabels)
    ax.set_xticks(ticklabels)
    ax.set_yticks(ticklabels)

This is very interesting, the word length matters in the last layers the most!

In [59]:
DSM_pixelwise_len = mne_rsa.compute_dsm(images_len, metric='euclidean')

In [60]:
rsa_len = mne_rsa.rsa(dsms_values, DSM_pixelwise_len, metric='kendall-tau-a')
print(rsa_len)

[ 0.88571429  0.73333333  0.54285714  0.25714286  0.25714286  0.37142857
 -0.06666667  0.56190476]


Solid result, but shoudn't RSA result be absolute val?

# Summary of RSA analysis

In [68]:
layers = ["conv1", "conv2", "conv3", "conv4", "dense1", "dense2", "lexicon", "semantic"]
rsas = [rsa_noise, rsa_font, rsa_rot, rsa_len]

In [70]:
fig, axes = plt.subplots(4, sharex=True, sharey=True, figsize=(8, 8))
titles = ["Noise", "Font", "Rotation", "Word Length"]
fig.suptitle("RSA analysis")

for i in range(4):
    print(rsas[i].shape)
    axes[i].bar(x=layers, height=rsas[i])
    axes[i].set_title(titles[i], fontdict={'verticalalignment': 'center'}, rotation='vertical', x=-0.125, y=0.5)
plt.setp(axes, ylim=(-0.2, 1));
plt.savefig("rsa_analysis.pdf")

(8,)
(8,)
(8,)
(8,)


In [90]:
fig, axes = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(5, 10))
titles = ["Noise", "Font", "Rotation", "Word Length"]
fig.suptitle("RSA analysis")

for i, layer in enumerate(zip(*rsas)):
    print(layer)
    axes.ravel()[i].bar(x=titles, height=layer)
    #axes[i].set_title(titles[i], fontdict={'verticalalignment': 'center'}, rotation='vertical', x=-0.125, y=0.5)
plt.setp(axes, ylim=(-0.2, 1));

(0.9097731813997726, 0.6084249084249084, 0.8088578088578089, 0.8857142857142857)
(0.655850809570753, 0.5142857142857142, 0.7202797202797203, 0.7333333333333333)
(0.6111181730417939, 0.47985347985347987, 0.6578088578088578, 0.5428571428571428)
(0.5692287596106549, 0.5333333333333333, 0.6587412587412588, 0.2571428571428571)
(0.5661415937512119, 0.2868131868131868, 0.5365967365967366, 0.2571428571428571)
(0.5631336194843974, 0.20695970695970695, 0.38181818181818183, 0.37142857142857144)
(0.3255181844715084, 0.25677655677655675, 0.10862470862470862, -0.06666666666666667)
(0.2549341464758721, 0.2562271062271062, 0.08438228438228439, 0.5619047619047619)


In [75]:
rsas

[array([0.90977318, 0.65585081, 0.61111817, 0.56922876, 0.56614159,
        0.56313362, 0.32551818, 0.25493415]),
 array([0.60842491, 0.51428571, 0.47985348, 0.53333333, 0.28681319,
        0.20695971, 0.25677656, 0.25622711]),
 array([0.80885781, 0.72027972, 0.65780886, 0.65874126, 0.53659674,
        0.38181818, 0.10862471, 0.08438228]),
 array([ 0.88571429,  0.73333333,  0.54285714,  0.25714286,  0.25714286,
         0.37142857, -0.06666667,  0.56190476])]